In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import pandas as pd
from collections import defaultdict
import numpy as np
import time
from multiprocessing import Process, Manager, Pool

In [4]:
button_dict = {'mail' : 'ejmi65789@google.com',
        'phone' : '01044444444',
        'name' : '이조용',
         'pw' : 'Kus410028!',
         'last_n' : 'lee',
         'first_n' : 'chong',
         'year' : '1994',
         'day' : '21',
         'month' : '6'}

In [7]:
result_dictionary = Manager().dict()
tern = 0
file = pd.read_csv('C:\\Users\\dlckd\\onedrive\\Desktop\\url.csv')
url_link = file['url']
url_link = np.array(url_link)
url_link = [link.lower().replace(' ', '') for link in url_link]
url_link = list(set(url_link))

In [8]:
next_word = ['다음', 'next', '시작', 'over', 'Next', 'NEXT', 'OVER', 'Over', '계속', 'continue', 'Continue', 'CONTINUE']
banner = ['cookie', 'Cookie', '쿠키', 'COOKIE', 'ACCEPT', 'Accept', 'Agree', 'Agree', '계속', 'Continue', 'continue', 'CONTINUE']
accept_word = ['accept', '동의', 'consent', 'sign up', 'register', '수락', 'ACCEPT', 'Accept', 'create', 'Create', 'CREATE', 'Sign Up', 'SIGN UP', '코드', 'code', 'CODE', 'Code', 'agree', 'Agree', 'AGREE', 'Sign up', 'Submit', 'SUBMIT', 'submit', '등록']
sign_up_word = ['회원가입', 'sign up', 'Sign UP', 'SIGN UP', 'create', 'CREATE', 'Create', 'Join', 'join', 'JOIN', 'signup', '계정 만들기', '등록', 'Sign up', 'SignUp', 'Try', 'TRY', 'try', 'register', 'Register', 'REGISTER', 'Get Start', 'GET START', 'get start']
privacy = ['privacy', '개인정보', '데이터정책', 'PRIVACY', 'Privacy', '프라이버시']
terms_of_use = ['use', 'agreement', '약관', 'Use', 'USE', 'AGREEMENT', 'Agreement', 'Terms', 'terms', 'TERMS']
total = list()
for p in privacy:
    for u in terms_of_use:
        total.append((p, u))

In [9]:
def link_check(tag, option):
    result = dict()
    if 'href' in tag.attrs:
        result[option+'_link']  = 'O'
    else:
        result[option+'_link']  = 'X'
    return result

def check_box(tag, option):
    result = dict()
    if 'input' in tag.attrs:
        if tag.attrs['input'] == 'check_box':
            result[option+'_check_box'] = 'O'
    elif tag.find('i'):
        result[option+'_check_box'] = 'O'
    elif 'before' in str(tag).lower():
        result[option+'_check_box'] = 'O'
    else:
        result[option+'_check_box'] = 'X'
    return result

def refine(tag, string):
    temporary = str(tag)
    if 'class="' in str(tag):
        temporary = temporary.split('class="')[1]
        temporary = temporary.split('"')
        string['class'] = temporary[0]
    temporary = str(tag)
    if 'id="' in str(tag):
        temporary = temporary.split('id="')[1]
        temporary = temporary.split('"')
        string['id'] = temporary[0]
    temporary = str(tag)
    if 'name="' in str(tag):
        temporary = temporary.split('name="')[1]
        temporary = temporary.split('"')
        string['name'] = temporary[0]
    temporary = str(tag)
    if 'href="' in str(tag):
        temporary = temporary.split('href="')[1]
        temporary = temporary.split('"')
        string['href'] = temporary[0]
        
def finally_refined(object_list):
    result = list()
    for obj in object_list:
        if obj not in result:
            result.append(obj)
    return result

class find_values:
    def __init__(self, data_direction, driver):
        self.value = data_direction
        self.drvier = driver
        
    def return_text(self, child):
        try:
            return child.text
        except:
            return child.string
        
    def dfs(self, start, next_link_dict):
        visited = list()
        queue = list()
        queue.append(start)
        
        while queue:
            node = queue.pop(0)
            if node not in visited:
                visited.append(node)
                try:
                    for child in list(node.children):
                        queue.append(child)
                        text = self.return_text(child)
                        for word in self.value:
                            if word in text:
                                temporary = dict()
                                refine(child, temporary)
                                if temporary:
                                    next_link_dict.append(temporary)
                                    #temporary['word'] = word
                                    
                        if 'value="' in str(child):
                            string = str(child).split('value="')[1]
                            string = string.split('"')[0]
                            for word in self.value:
                                if word in string:
                                    temporary = dict()
                                    refine(child, temporary)
                                    if temporary:
                                        next_link_dict.append(temporary)
                                        #temporary['word'] = word
                except : pass 
                
    def create_link(self, links):
        driver = self.drvier
        for link in reversed(links):
            if 'href' not in link:
                continue
            if 'help' in link['href']:
                continue
            
            just_href_check = True
            
            for idx, word in enumerate(self.value):
                if word in link['href']:
                    break
                if idx == len(self.value)-1:
                    just_href_check = False
                          
            if just_href_check == False:
                continue
                
            before_soup = BeautifulSoup(driver.page_source, 'lxml')
            driver.get(link['href'])
            time.sleep(5)
            after_soup = BeautifulSoup(driver.page_source, 'lxml')
            if after_soup != before_soup:
                return
            
        for link in reversed(links):
            before_soup = BeautifulSoup(driver.page_source, 'lxml')
            trial(None, link, driver)
            time.sleep(5)
            after_soup = BeautifulSoup(driver.page_source, 'lxml')
            if after_soup != before_soup:
                return
        
    def policy_dfs(self, start, option):
        visited = list()
        queue = list()
        queue.append(start)
        count = 0
        if option == 1:
            while queue:
                node = queue.pop(0)
                if node not in visited:
                    visited.append(node)
                    try:
                        for child in list(node.children):
                            text = self.return_text(child)
                            for word_1, word_2 in self.value:
                                if word_1 in text and word_2 in text:
                                    queue.append(child)

                            if 'value="' in str(child):
                                string = str(child).split('value="')[1]
                                string = string.split('"')[0]
                                for word_1, word_2 in self.value:
                                    if word_1 in string and word_2 in string:
                                        queue.append(child)
                    except: pass
        else:
            while queue:
                node = queue.pop(0)
                if node not in visited:
                    visited.append(node)
                    try:
                        for child in list(node.children):
                            text = self.return_text(child)
                            for word_1 in self.value:
                                if word_1 in text:
                                    queue.append(child)

                            if 'value="' in str(child):
                                string = str(child).split('value="')[1]
                                string = string.split('"')[0]
                                for word_1 in self.value:
                                    if word_1 in string:
                                        queue.append(child)
                    except: pass

        return visited
    
    def find_tag(self, target):
        temp = target.pop()
        while 'NavigableString' in str(type(temp)) and target:
            temp = target.pop()
            if 'NavigableString' not in str(type(temp)):
                if temp.parent.find('footer'):
                    continue
                else:
                    break
        return temp

def select(soup, driver):
    for sl in soup.find_all('select'):
        select_tags = {}
        select_check = str(sl).lower()
        if 'country' in select_check or '국가' in select_check:
            select_trial('country', sl, driver)
        elif '년' in select_check or 'year' in select_check or '해' in select_check:
            select_trial('year', sl, driver)
        elif '달' in select_check or 'month' in select_check or '월' in select_check:
            select_trial('month', sl, driver)
        elif 'day' in select_check or '일' in select_check or '날' in select_check:
            select_trial('day', sl, driver)
        elif 'city' in select_check or '도시' in select_check or '도' in select_check:
            select_trial('city', sl, driver)
            
def input_(soup, check, driver):
    for ipt in soup.find_all(['input']):
        input_tags = {}
        input_check = str(ipt).lower()
        if 'autocomplete' in input_check or 'aria-' in input_check:
            if 'phone' in input_check and check['phone']!=True:
                refine(ipt, input_tags)
                trial('phone', input_tags, driver)
                check['phone'] = True
            elif 'password' in input_check and check['password']!=True:
                refine(ipt, input_tags)
                trial('pw', input_tags, driver)
                check['password'] = True
            elif 'last' in input_check and check['last']!=True:
                refine(ipt, input_tags)
                trial('last_n', input_tags, driver)
                check['last'] = True
            elif 'first' in input_check and check['first']!=True or 'family' in input_check:
                refine(ipt, input_tags)
                trial('first_n', input_tags, driver)
                check['first'] = True
            elif 'day' in input_check and check['day']!=True:
                check['day'] = True
                refine(ipt, input_tags)
                trial('day', input_tags, driver)
            elif 'month' in input_check and check['month']!=True:
                check['month'] = True
                refine(ipt, input_tags)
                trial('month', input_tags)
            elif 'year' in input_check and check['year']!=True:
                check['year'] = True
                refine(ipt, input_tags)
                trial('year', input_tags, driver)
            elif 'email' in input_check and check['email']!= True:
                refine(ipt, input_tags)
                trial('mail', input_tags, driver) 
                check['email'] = True
            elif 'name' in input_check and check['name'] != True:
                refine(ipt, input_tags)
                trial('name', input_tags, driver)
                check['name'] = True
    return check

def select_trial(opt, string, driver):
    if opt == 'country':
        try:
            driver.find_element_by_xpath("//option[@value='"+'KR'+"']").click()
        except:
            pass
    elif opt == 'year':
        try:
            driver.find_element_by_xpath("//option[@value='"+'2000'+"']").click()
        except:
            pass
    elif opt == 'month':
        try:
            driver.find_element_by_xpath("//option[@value='"+'6'+"']").click()
        except : pass
    elif opt == 'day':
        try:
            driver.find_element_by_xpath("//option[@value='"+'20'+"']").click()
        except:
            pass
    elif opt == 'city':
        temporary = str(string).split('</option>')[2]
        temporary = temporary.split('value="')[1]
        temporary = temporary.split('"')[0]
        try:
            driver.find_element_by_xpath("//option[@value='"+temporary+"']").click()
        except: pass

def trial(attrs, string, driver):
    if attrs!=None:
        try:
            driver.find_element_by_id(string['id']).send_keys(button_dict[attrs])
        except:
            try:
                driver.find_element_by_name(string['name']).send_keys(button_dict[attrs])
            except:
                try:
                    driver.find_element_by_class_name(string['class']).send_keys(button_dict[attrs])
                except Exception as e:
                    return e
    else:
        try:
            driver.find_element_by_class_name(string['class']).click()
        except:
            try:
                driver.find_element_by_id(string['id']).click()
            except:
                try:
                    driver.find_element_by_name(string['name']).click()
                except Exception as e:
                    return 'error'

In [10]:
def practice(url):
    global tern
    global result_dictionary
    
    tern+=1
    print(url, end = ' ')

    if 'netflix' in url or 'google' in url or 'openload' in url or 'vk' in url:
        result_dictionary[url]['link_term'] = 'test on mechanic',
        result_dictionary[url]['term_check_box'] = 'test on mechanic',
        result_dictionary[url]['link_privacy'] = 'test on mechanic',
        result_dictionary[url]['privacy_check_box'] = 'test on mechanic',
        result_dictionary[url]['exist'] = 'test on mechanic'
        driver.quit()

    driver = webdriver.Chrome('C:\\Users\\dlckd\\onedrive\\Desktop\\chromedriver.exe')
    driver.get('https://www.vpnbook.com/webproxy')
    driver.implicitly_wait(20)
    time.sleep(20)
    driver.find_element_by_xpath('//*[@id="input"]').send_keys('https://www.google.co.uk/webhp?hl=en')
    driver.find_element_by_xpath('//*[@id="webproxylocation"]/option[3]').click()
    driver.find_element_by_xpath('//*[@id="webproxyform"]/input[2]').click()
    driver.implicitly_wait(20)
    time.sleep(7)
    driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div[1]/div[1]/div/div[2]/input').send_keys(url+' sign up')
    driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div[1]/div[3]/center/input[1]').click()
    driver.implicitly_wait(20)
    time.sleep(10)

    try:
        driver.find_element_by_xpath('//*[@id="rso"]/div[1]/div[1]/div/div[1]/a/h3').click()
    except:
        try:
            driver.find_element_by_xpath('//*[@id="rso"]/div[2]/div/div/div[1]/a/h3').click()
        except:
            try:
                driver.find_element_by_xpath('//*[@id="rso"]/div/div[1]/div/div[1]/a/h3').click()
            except:
                result_dictionary[url]['link_term'] = 'xpth-check',
                result_dictionary[url]['term_check_box'] = 'xpth-check',
                result_dictionary[url]['link_privacy'] = 'xpth-check',
                result_dictionary[url]['privacy_check_box'] = 'xpth-check',
                result_dictionary[url]['exist'] = 'xpth-check'
                driver.quit()
                
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    banner_find = find_values(banner, driver)
    banner_list = list()
    banner_find.dfs(soup, banner_list)
    banner_list = finally_refined(banner_list)
    print(banner_list)
    for btn in banner_list:
        before = BeautifulSoup(driver.page_source, 'lxml')
        if 'href' in btn:
            continue
        trial(None, btn, driver)
        time.sleep(7)
        after = BeautifulSoup(driver.page_source, 'lxml')
        if after!=before:
            break

    driver.implicitly_wait(20)
    time.sleep(10)
    soup = BeautifulSoup(driver.page_source, 'lxml')

    input_check = {'phone' : False,
     'password' : False,
     'last' : False,
     'first' : False,
     'day' : False,
     'month' : False,
     'year' : False,
     'name' : False,
     'email' : False}

    input_(soup, input_check, driver)

    count_input = len([x for x in input_check if input_check[x]==True])

    if count_input < 3:
        find_join_link = find_values(sign_up_word, driver)
        join_link = list()
        find_join_link.dfs(soup, join_link)
        join_link = finally_refined(join_link)
        for_join_link = find_join_link.create_link(join_link)
        time.sleep(7)
        soup = BeautifulSoup(driver.page_source, 'lxml')

    print(len(soup.find_all('body')))

    check_soup = 1
    count = 1

    input_check = {'phone' : False,
     'password' : False,
     'last' : False,
     'first' : False,
     'day' : False,
     'month' : False,
     'year' : False,
     'name' : False,
     'email' : False}

    while check_soup !=soup and 7>=count:

        test = find_values(next_word, driver)
        temp = soup.body
        link_next = list()
        test.dfs(temp, link_next)
        if not link_next or len(link_next) == 0:
            break

        link_next = finally_refined(link_next)
        before_input = input_check
        if check_soup != 1:
            soup = check_soup
        if soup.find(['select']):
            select(soup, driver)
        if soup.find(['input']):
            input_(soup, input_check, driver)
        check = True

        while link_next:
            pos = link_next.pop()
            if 'href' in pos:
                if pos['href'] == 'javascript:void(0);':
                    continue
            message = trial(None, pos, driver)
            if message == 'error':
                continue
            time.sleep(5)
            if 1<len(driver.window_handles):
                driver.switch_to.window(driver.window_handles[0])
            temp_soup = BeautifulSoup(driver.page_source, 'lxml')
            if temp_soup!=soup:
                break
            if not link_next and before_input!=input_check:
                check = False

        if check == False:
            count = None
            break

        check_soup = BeautifulSoup(driver.page_source, 'lxml')
        count +=1
        time.sleep(7)

    if count == None:
        result_dictionary[url]['direct'] = 'cannot go next'
        result_dictionary[url]['term_link'] = 'cannot go next'
        result_dictionary[url]['term_check_box'] = 'cannot go next'
        result_dictionary[url]['privacy_link'] = 'cannot go next'
        result_dictionary[url]['privacy_check_box'] = 'cannot go next'
        result_dictionary[url]['exist'] = 'cannot go next'
        driver.quit()

    soup = BeautifulSoup(driver.page_source, 'lxml')
    accept_find = find_values(accept_word, driver)
    accept_list = list()
    accept_find.dfs(soup, accept_list)
    accept_list = finally_refined(accept_list)

    if not accept_list:
        result_dictionary[url]['direct'] = 'no-accept-button'
        result_dictionary[url]['term_link'] = 'no-accept-button',
        result_dictionary[url]['term_check_box'] = 'no-accept-button',
        result_dictionary[url]['privacy_link'] = 'no-accept-button',
        result_dictionary[url]['privacy_check_box'] = 'no-accept-button',
        result_dictionary[url]['exist'] = 'no-accept-button'
        driver.quit()

    privacy_find = find_values(privacy, driver)
    privacy_list = privacy_find.policy_dfs(soup, 2)
    use_find = find_values(terms_of_use, driver)
    use_list = use_find.policy_dfs(soup, 2)
    privacy_list = finally_refined(privacy_list)
    use_list = finally_refined(use_list)
    total_find = find_values(total, driver)
    x = total_find.policy_dfs(soup, 1)
    x = finally_refined(x)
    y = privacy_find.find_tag(privacy_list)
    z = use_find.find_tag(use_list)
    x = total_find.find_tag(x)

    if x.find('head') or x.find('title'):
        result_dictionary[url]['direct'] = 'O'
        result_dictionary[url]['term_link'] = 'X'
        result_dictionary[url]['term_checkbox'] = 'X'
        result_dictionary[url]['privacy_link'] = 'X'
        result_dictionary[url]['privacy_check_box'] = 'X'
        result_dictionary[url]['exist_check_box'] = 'X'

    if 'direct' not in result_dictionary[url]:
        result_dictionary[url]['direct'] = 'X'
        if x == y.parent or x.parent == y.parent:
            for_append = check_box(x, 'exist')
            for key, value in for_append.items():
                result_dictionary[url][key] = value
            result_dictionary[url]['privacy_check_box'] = 'X'
            result_dictionary[url]['term_check_box'] = 'X'
            before_z = z
            for_new_z = find_values(terms_of_use, driver)
            temporary = for_new_z.policy_dfs(y.parent, 2)
            new_z = for_new_z.find_tag(temporary)
            if before_z == new_z:
                new_z = for_new_z.find_tag(temporary)
            for_append = link_check(z, 'term')
            for key, value in for_append.items():
                result_dictionary[url][key] = value
            for_append = link_check(y, 'privacy')
            for key, value in for_append.items():
                result_dictionary[url][key] = value

        elif x == z.parent or x.parent == z.parent:
            result_dictionary[url]['direct'] = 'X'
            for_append = check_box(x, 'exist')
            for key, value in for_append.items():
                result_dictionary[url][key] = value
            result_dictionary[url]['privacy_check_box'] = 'X'
            result_dictionary[url]['term_check_box'] = 'X'
            before_y = y
            for_new_y = find_values(privacy, driver)
            temporary = for_new_y.policy_dfs(z.parent, 2)
            new_y = for_new_y.find_tag(temporary)
            if before_y == new_y:
                new_y = for_new_y.find_tag(temporary)
            for_append = link_check(z, 'term')
            for key, value in for_append.items():
                result_dictionary[url][key] = value
            for_append = link_check(new_y, 'privacy')
            for key, value in for_append.items():
                result_dictionary[url][key] = value

        else:
            result_dictionary[url]['exist_check_box'] = 'X'
            for_append = link_check(y, 'term')
            for key, value in for_append.items():
                result_dictionary[url][key] = value
            for_append = link_check(z, 'privacy')
            for key, value in for_append.items():
                result_dictionary[url][key] = value
            for_append = check_box(y.parent, 'term')
            for key, value in for_append.items():
                result_dictionary[url][key] = value
            for_append = check_box(z.parent, 'privacy')
            for key, value in for_append.items():
                result_dictionary[url][key] = value
    driver.quit()
    if tern%10 == 0:
        df = pd.DataFrame(result_dictionary)
        df.to_csv('C:\\Users\\dlckd\\onedrive\\Desktop\\'+str(tern)+url+'.csv')

In [ ]:
if __name__ == '__main__':
    pool = Pool(processes = 4)
    pool.map(practice, url_link)